### Runnable with Message History

[Agent Executor - Adding in memory](https://python.langchain.com/docs/how_to/agent_executor/#adding-in-memory)  
[API reference - Runnable with Message History](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html)  
[API reference - BaseChatMessageHistory](https://python.langchain.com/api_reference/core/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html)

In [1]:
from dotenv import load_dotenv
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
import os
import sys
load_dotenv()

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from tool_utils import naver_news, naver_dict # 반드시 .env파일 불러온 이후 import하기

###  1. 기본 AgentExecutor 정의

In [2]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 'You are a helpful assistant'),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}") # MUST

    ]
)
model = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [naver_news, naver_dict]
agent = create_tool_calling_agent(model, tools, prompt) 
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True) # -> chain

### 2. Chat history 정의 및 Agent Executor에 chat history를 붙인 chain 만들기

In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
store = dict()

def get_by_session_history(session_id: str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor, # agentexecutor도 일종의 Runnable 객체이므로 RunnableWithMessageHistory의첫 parameter로 들어갈 수 있음
    get_by_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

### 3. 대화 진행

In [5]:
# 첫 번째 턴
to_invoke:dict = {"input": "난 경제를 잘 알고 싶어. 20자 이내로 짧게 대답해줘."}

for step in agent_with_chat_history.stream(to_invoke, config={'configurable':{"session_id": "foo"}}):
    step["messages"][-1].pretty_print()



> Entering new None chain...
경제는 자원 배분과 거래의 학문입니다.

> Finished chain.
================================== Ai Message ==================================

경제는 자원 배분과 거래의 학문입니다.


In [6]:
# 두 번째 턴
to_invoke:dict = {"input": "내가 뭘 잘 알고 싶다고 했어?"}

for step in agent_with_chat_history.stream(to_invoke, config={'configurable':{"session_id": "foo"}}):
    step["messages"][-1].pretty_print()



> Entering new None chain...
경제를 잘 알고 싶다고 하셨습니다.

> Finished chain.
================================== Ai Message ==================================

경제를 잘 알고 싶다고 하셨습니다.


### 저장된 Chat History 확인

In [7]:
store

{'foo': InMemoryChatMessageHistory(messages=[HumanMessage(content='난 경제를 잘 알고 싶어. 20자 이내로 짧게 대답해줘.', additional_kwargs={}, response_metadata={}), AIMessage(content='경제는 자원 배분과 거래의 학문입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='내가 뭘 잘 알고 싶다고 했어?', additional_kwargs={}, response_metadata={}), AIMessage(content='경제를 잘 알고 싶다고 하셨습니다.', additional_kwargs={}, response_metadata={})])}